In [131]:
import os
import imageio

In [132]:
import pandas as pd
import numpy as np

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Scatter, Figure, Layout
import plotly.graph_objs as go
import plotly.plotly as py
from plotly.plotly import image as p_image

%matplotlib inline
init_notebook_mode(connected=True)

# Data

In [140]:
import os
reactions_df = pd.DataFrame()
for filename in os.listdir('./result_emotions/'):
    reaction_df = pd.read_csv('./result_emotions/' + filename)
    reaction_df['id'] = filename
    reaction_df['time_second'] = [x.split('.')[0] for x in reaction_df['time'].astype('str').values]
    reaction_df['time_second'] = reaction_df['time_second'].astype('int')
    reactions_df = reactions_df.append(reaction_df)

In [144]:
nrd = pd.DataFrame()
for id_f, frame_df in reactions_df.groupby(['id']):
    f_df = frame_df.copy()
    
    f_df_sec = f_df.groupby(['id', 'time_second'], as_index = False)['reaction'].agg(lambda x:x.value_counts().index[0])
    f_df_sec = f_df_sec.merge(f_df[['id',  'gender', 'time_second', 'reaction', 'reaction_tense']], on = ['id', 'time_second', 'reaction']).groupby(['id',  'gender', 'time_second', 'reaction'], as_index = False)['reaction_tense'].mean()
    
    f_df_sec = f_df_sec.tail(69)
    f_df_sec.time_second = range(f_df_sec.shape[0])
    
    nrd = nrd.append(f_df_sec)

# Visualize

In [146]:
reaction_df = nrd.copy() # reaction_df_sec.copy()

In [147]:
# settings
max_second = reaction_df.time_second.max()
first_second = reaction_df_sec.shape[0]

In [152]:
# singe frame
new_reaction_df = pd.DataFrame()
for id_f, frame_df in reaction_df.groupby(['id']):
    first_second = frame_df.shape[0]
    frame_df = frame_df[:first_second].copy()
    frame_df['graph_tense'] = frame_df['reaction_tense']
    frame_df.loc[frame_df[frame_df.reaction == 'surprise'].index, 'reaction'] = 'neutral'
    frame_df.loc[frame_df[frame_df.reaction == 'sad'].index, 'reaction'] = 'angry'
    frame_df.loc[frame_df[frame_df.reaction == 'happy'].index, 'graph_tense'] = frame_df[frame_df.reaction == 'happy']['graph_tense'] * 33 + 66
    frame_df.loc[frame_df[frame_df.reaction == 'neutral'].index, 'graph_tense'] = frame_df[frame_df.reaction == 'neutral']['graph_tense'] * 33 + 33
    frame_df.loc[frame_df[frame_df.reaction == 'angry'].index, 'graph_tense'] = frame_df[frame_df.reaction == 'angry']['graph_tense'] * 33

    new_reaction_df = new_reaction_df.append(frame_df)

In [153]:
new_reaction_df['id'] = new_reaction_df['id'].astype('str')
new_reaction_df.reset_index(inplace=True, drop = True)

In [154]:
# male vs female
new_reaction_df= new_reaction_df.groupby(['time_second', 'gender'], as_index=False)['graph_tense'].mean()
new_reaction_df.loc[new_reaction_df[(new_reaction_df['graph_tense'] < 33)].index, 'reaction'] = 'angry'
new_reaction_df.loc[new_reaction_df[(new_reaction_df['graph_tense'] >= 33) & (new_reaction_df['graph_tense'] < 66)].index, 'reaction'] = 'neutral'
new_reaction_df.loc[new_reaction_df[(new_reaction_df['graph_tense'] >= 66)].index, 'reaction'] = 'happy'
new_reaction_df.loc[new_reaction_df[new_reaction_df.gender == 'male'].index, 'color'] = 'rgb(17,56,117)'
new_reaction_df.loc[new_reaction_df[new_reaction_df.gender == 'female'].index, 'color'] = 'rgb(166,28,0)'

new_reaction_df.head()

,time_second,gender,graph_tense,reaction,color
0,0,female,56.398291,neutral,"rgb(166,28,0)"
1,0,male,59.050990,neutral,"rgb(17,56,117)"
2,1,female,57.248429,neutral,"rgb(166,28,0)"
3,1,male,57.129029,neutral,"rgb(17,56,117)"
4,2,female,56.312193,neutral,"rgb(166,28,0)"


In [128]:
# # all-in-one
# new_reaction_df= new_reaction_df.groupby('time_second', as_index=False)['graph_tense'].mean()
# new_reaction_df.loc[new_reaction_df[(new_reaction_df['graph_tense'] < 33)].index, 'reaction'] = 'angry'
# new_reaction_df.loc[new_reaction_df[(new_reaction_df['graph_tense'] >= 33) & (new_reaction_df['graph_tense'] < 66)].index, 'reaction'] = 'neutral'
# new_reaction_df.loc[new_reaction_df[(new_reaction_df['graph_tense'] >= 66)].index, 'reaction'] = 'happy'
# new_reaction_df['color'] = 'rgb(0, 0, 0)'
# new_reaction_df.head()

,time_second,graph_tense,reaction,color
0,0,58.166757,neutral,"rgb(0, 0, 0)"
1,1,57.168829,neutral,"rgb(0, 0, 0)"
2,2,56.051501,neutral,"rgb(0, 0, 0)"
3,3,56.240176,neutral,"rgb(0, 0, 0)"
4,4,56.085051,neutral,"rgb(0, 0, 0)"


In [155]:
x = sorted(new_reaction_df.time_second.unique())
y4 = [0 for x in range(frame_df.shape[0])]
y1 = [33 for x in range(frame_df.shape[0])]
y2 = [66 for x in range(frame_df.shape[0])]
y3 = [100 for x in range(frame_df.shape[0])]

trace1 = go.Scatter(
    x=x,
    y=y1,
    fill='tozeroy',
    line=dict(
        width=0.5,
        color='rgb(230,184,175)',
    )
)
trace2 = go.Scatter(
    x=x,
    y=y2,
    fill='tonexty',
    opacity=0.5,
    line = dict(
        width=0.5,
        color='rgb(255,242,204)'
    )
)
trace3 = go.Scatter(
    x=x,
    y=y3,
    fill='tonexty',
    opacity=0.5,
    line=dict(
        width=0.5,
        color='rgb(182,215,168)',
    )
)
trace4 = go.Scatter(
    x=x,
    y=y4,
    fill='tonexty',
    opacity=0.5,
    line=dict(
        width=0.5,
        color='rgb(230,184,175)',
    )

)
data = [trace4, trace1, trace2, trace3]

for id_f, frame_df in new_reaction_df.groupby(['gender']):
    
    trace = go.Scatter(
        x = frame_df.time_second.values,
        y = frame_df.graph_tense.values,
#         mode = 'lines+markers',
        line=dict(
            width=3,
            color=frame_df.color.values[0],
        )
    )
    
    data.append(trace)

# trace = go.Scatter(
#         x = new_reaction_df.time_second.values,
#         y = new_reaction_df.graph_tense.values,
#         line=dict(
#             width=3,
#             color=new_reaction_df.color.values[0],
#         )
#     )
    
# data.append(trace)
    
layout = go.Layout(
    showlegend=False,
    yaxis=dict(
        type='linear',
        range=[0, 100],
    )
) 

fig = go.Figure(data=data, layout=layout)
plot(fig, image='png', image_filename='background', filename='graphs/background.html')

'file://C:\\Users\\ndmel\\Desktop\\funanalytics\\Emotion-master\\Emotion-master\\graphs\\background.html'

# Save series of images

In [156]:
# settings
x = sorted(reaction_df_sec.time_second.unique())
for first_second in range(1, reaction_df_sec.time_second.max() + 1):
    
    filename = 'frame_%s' % str(first_second)
    
    # singe frame
    new_reaction_df = pd.DataFrame()
    for id_f, frame_df in reaction_df.groupby(['id']):
        frame_df = frame_df[:first_second].copy()
        frame_df['graph_tense'] = frame_df['reaction_tense']
        frame_df.loc[frame_df[frame_df.reaction == 'surprise'].index, 'reaction'] = 'neutral'
        frame_df.loc[frame_df[frame_df.reaction == 'sad'].index, 'reaction'] = 'angry'
        frame_df.loc[frame_df[frame_df.reaction == 'happy'].index, 'graph_tense'] = frame_df[frame_df.reaction == 'happy']['graph_tense'] * 33 + 66
        frame_df.loc[frame_df[frame_df.reaction == 'neutral'].index, 'graph_tense'] = frame_df[frame_df.reaction == 'neutral']['graph_tense'] * 33 + 33
        frame_df.loc[frame_df[frame_df.reaction == 'angry'].index, 'graph_tense'] = frame_df[frame_df.reaction == 'angry']['graph_tense'] * 33

        new_reaction_df = new_reaction_df.append(frame_df)
        
    new_reaction_df['id'] = new_reaction_df['id'].astype('str')
    new_reaction_df.reset_index(inplace=True, drop = True)
#     new_reaction_df= new_reaction_df.groupby('time_second', as_index=False)['graph_tense'].mean()
#     new_reaction_df.loc[new_reaction_df[(new_reaction_df['graph_tense'] < 33)].index, 'reaction'] = 'angry'
#     new_reaction_df.loc[new_reaction_df[(new_reaction_df['graph_tense'] >= 33) & (new_reaction_df['graph_tense'] < 66)].index, 'reaction'] = 'neutral'
#     new_reaction_df.loc[new_reaction_df[(new_reaction_df['graph_tense'] >= 66)].index, 'reaction'] = 'happy'
#     new_reaction_df['color'] = 'rgb(0, 0, 0)'
#     new_reaction_df.head()
    
    
    new_reaction_df= new_reaction_df.groupby(['time_second', 'gender'], as_index=False)['graph_tense'].mean()
    new_reaction_df.loc[new_reaction_df[(new_reaction_df['graph_tense'] < 33)].index, 'reaction'] = 'angry'
    new_reaction_df.loc[new_reaction_df[(new_reaction_df['graph_tense'] >= 33) & (new_reaction_df['graph_tense'] < 66)].index, 'reaction'] = 'neutral'
    new_reaction_df.loc[new_reaction_df[(new_reaction_df['graph_tense'] >= 66)].index, 'reaction'] = 'happy'
    new_reaction_df.loc[new_reaction_df[new_reaction_df.gender == 'male'].index, 'color'] = 'rgb(17,56,117)'
    new_reaction_df.loc[new_reaction_df[new_reaction_df.gender == 'female'].index, 'color'] = 'rgb(166,28,0)'

    # plot
    y4 = [0 for x in range(len(x))]
    y1 = [33 for x in range(len(x))]
    y2 = [66 for x in range(len(x))]
    y3 = [100 for x in range(len(x))]

    trace1 = go.Scatter(
        x=x,
        y=y1,
        fill='tozeroy',
        line=dict(
            width=0.5,
            color='rgb(230,184,175)',
        )
    )
    trace2 = go.Scatter(
        x=x,
        y=y2,
        fill='tonexty',
        opacity=0.5,
        line = dict(
            width=0.5,
            color='rgb(255,242,204)'
        )
    )
    trace3 = go.Scatter(
        x=x,
        y=y3,
        fill='tonexty',
        opacity=0.5,
        line=dict(
            width=0.5,
            color='rgb(182,215,168)',
        )
    )
    trace4 = go.Scatter(
        x=x,
        y=y4,
        fill='tonexty',
        opacity=0.5,
        line=dict(
            width=0.5,
            color='rgb(230,184,175)',
        )

    )
    data = [trace4, trace1, trace2, trace3]
    
    for id_f, frame_df in new_reaction_df.groupby(['gender']):
    
        trace = go.Scatter(
            x = frame_df.time_second.values,
            y = frame_df.graph_tense.values,
    #         mode = 'lines+markers',
            line=dict(
                width=3,
                color=frame_df.color.values[0],
            )
        )

        data.append(trace)

#     trace = go.Scatter(
#         x = new_reaction_df.time_second.values,
#         y = new_reaction_df.graph_tense.values,
#         line=dict(
#             width=3,
#             color=new_reaction_df.color.values[0],
#         )
#     )
    
#     data.append(trace)

    layout = go.Layout(
        title='Анализ ролика',
        showlegend=False,
        xaxis=dict(
            title='Секунды',
            range=[min(x) - 1, max(x) + 1]
        ),
        yaxis=dict(
            title='Настроениие зрителя',
            type='linear',
            range=[0, 100],
            ticksuffix='%'
        ),
    ) 
            
    fig = go.Figure(data=data, layout=layout)
    plot(fig, image='png', image_filename=filename, filename='graphs/%s.html' % filename)

# Make gif

In [211]:
# images = []
# for filename in os.listdir('./gif_graphs/'):
#     images.append(imageio.imread(os.getcwd() + '\\gif_graphs\\' + filename))
# imageio.mimsave('result.gif', images)    